In [1]:
import pandas as pd
from pandas import DataFrame,Series
import numpy as np
import os
import datetime

#Plotting
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline

# sklearn stuff
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, mean_squared_error, precision_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, Imputer 
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

import feature_pipelines as pipes

### Reading in data

In [2]:
maindir = "/home/anerdi/Desktop/Zillow"

logerror = pd.read_csv(maindir + "/data/train_2016_v2.csv/train_2016_v2.csv")
logerror['weeknumber'] = logerror['transactiondate'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d').isocalendar()[1])
logerror['month'] = logerror['transactiondate'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d').month)
properties = pd.read_csv(maindir + "/data/properties_2016.csv/properties_2016.csv", usecols=['parcelid'])

In [3]:
# join on parcel id
data = pd.merge(properties,logerror[['parcelid','logerror','month']], on='parcelid')

In [4]:
def mean_absolute_errors(submission_df, comparison_df):
    """
    This function takes a submission entry for public leaderboard, and returns
    the training error for each month.
    """
    # training error
    trainresults = pd.merge(submission_df[['ParcelId','201610','201611','201612']], comparison_df[['parcelid','logerror','month']],
                           left_on='ParcelId', right_on='parcelid')
    oct_error = abs(trainresults[trainresults['month'] == 10]['201610'] 
                    - trainresults[trainresults['month'] == 10]['logerror']).mean()
    nov_error = abs(trainresults[trainresults['month'] == 11]['201611'] 
                    - trainresults[trainresults['month'] == 11]['logerror']).mean()
    dec_error = abs(trainresults[trainresults['month'] == 12]['201612'] 
                    - trainresults[trainresults['month'] == 12]['logerror']).mean()
    overall_mae = (oct_error*(trainresults['month'] == 10).sum() + nov_error*(trainresults['month'] == 11).sum() 
                        + dec_error*(trainresults['month'] == 12).sum()) / (trainresults['month'].isin([10,11,12])).sum()
    return (oct_error, nov_error, dec_error, overall_mae)

### Readin stage 1 classification results 

In [5]:
overestimate_probabilities = pd.read_csv("/home/anerdi/Desktop/Zillow/twostagemodel/overestimate_probs_stacked_ann_rfs_xgbs_lgbms.csv.gz",
                                        compression='gzip')
overestimate_probabilities.rename(columns={'stacked_pred':'overestimate_prob'},inplace=True)
overestimate_probabilities.head()

,parcelid,ann_overestimate_prob,rf2_overestimate_prob,rf3_overestimate_prob,xgb1_overestimate_prob,xgb2_overestimate_prob,lgbm1_overestimate_prob,lgbm2_overestimate_prob,overestimate_prob
0,10754147,0.469632,0.466012,0.495383,0.658496,0.665265,0.631455,0.705203,0.635402
1,10759547,0.406392,0.555562,0.524675,0.536781,0.519523,0.552883,0.673566,0.534260
2,10843547,0.880731,0.548264,0.548752,0.430116,0.485831,0.575907,0.620080,0.588496
3,10859147,0.569588,0.663067,0.543329,0.679050,0.547760,0.614516,0.657634,0.625219
4,10879947,0.540791,0.519636,0.485341,0.502730,0.516258,0.486744,0.364720,0.483828


### Readin two-stage linear model predictions 

In [6]:
two_stage_linear_models = pd.read_csv("/home/anerdi/Desktop/Zillow/twostagemodel/two_stage_preds_linear_models_age.csv.gz",
                       compression='gzip')
two_stage_linear_models.head()

,parcelid,ridge_10_under,ridge_11_under,ridge_12_under,ridge_10_over,ridge_11_over,ridge_12_over,enet_10_under,enet_11_under,enet_12_under,...,larm_12_under,larm_10_over,larm_11_over,larm_12_over,huber_10_under,huber_11_under,huber_12_under,huber_10_over,huber_11_over,huber_12_over
0,10754147,-0.083343,-0.083460,-0.083549,0.081872,0.081947,0.082185,-0.073147,-0.073147,-0.073147,...,-0.067742,0.073268,0.073268,0.073268,-0.030622,-0.030622,-0.030622,0.028524,0.028524,0.028524
1,10759547,-0.080735,-0.080853,-0.080942,0.080371,0.080446,0.080683,-0.073147,-0.073147,-0.073147,...,-0.067742,0.073268,0.073268,0.073268,-0.030622,-0.030622,-0.030622,0.028524,0.028524,0.028524
2,10843547,-0.820392,-0.820510,-0.820599,1.631166,1.631242,1.631479,-0.092616,-0.092616,-0.092616,...,-0.067742,0.073268,0.073268,0.073268,-0.117576,-0.117576,-0.117576,0.109521,0.109521,0.109521
3,10859147,-0.113977,-0.114095,-0.114183,0.109437,0.109512,0.109749,-0.098430,-0.098430,-0.098430,...,-0.071982,0.075447,0.075447,0.075447,-0.103229,-0.103229,-0.103229,0.096156,0.096156,0.096156
4,10879947,-0.094550,-0.094668,-0.094757,0.091331,0.091406,0.091643,-0.083955,-0.083955,-0.083955,...,-0.072357,0.075640,0.075640,0.075640,-0.035014,-0.035014,-0.035014,0.032615,0.032615,0.032615


### Readin original linear models (i.e., one stage models) 

In [7]:
ridge = pd.read_csv("/home/anerdi/Desktop/Zillow/submissions/Ridge.gz")
lasso = pd.read_csv("/home/anerdi/Desktop/Zillow/submissions/Lasso.gz")
enet = pd.read_csv("/home/anerdi/Desktop/Zillow/submissions/ElasticNet.gz")
larm = pd.read_csv("/home/anerdi/Desktop/Zillow/submissions/LARM.gz")
huber = pd.read_csv("/home/anerdi/Desktop/Zillow/submissions/Huber_noweight.gz")

In [8]:
models = [
    ('ridge', ridge),
    ('lasso', lasso),
    ('enet', enet),
    ('larm', larm),
    ('huber', huber)
         ]

### Combine preds & overestimate probabilities to generate one prediction 

In [9]:
overestimate_probabilities.overestimate_prob.describe()

count    2.985217e+06
mean     5.658707e-01
std      7.666480e-02
min      1.837959e-01
25%      5.201498e-01
50%      5.699797e-01
75%      6.175720e-01
max      8.113507e-01
Name: overestimate_prob, dtype: float64

In [10]:
test_predictions = pd.merge(two_stage_linear_models, overestimate_probabilities, on='parcelid')

In [11]:
for model_pair in models:
    current_model_name, single_stage_model_results = model_pair
    
    for month in [10,11,12]:
        test_predictions['%s_%d' % (current_model_name, month)] = (
                test_predictions['%s_%d_over' % (current_model_name, month)]*test_predictions['overestimate_prob'] 
                + test_predictions['%s_%d_under' % (current_model_name, month)]*(1 - test_predictions['overestimate_prob']))

In [12]:
test_predictions.head()

,parcelid,ridge_10_under,ridge_11_under,ridge_12_under,ridge_10_over,ridge_11_over,ridge_12_over,enet_10_under,enet_11_under,enet_12_under,...,lasso_12,enet_10,enet_11,enet_12,larm_10,larm_11,larm_12,huber_10,huber_11,huber_12
0,10754147,-0.083343,-0.083460,-0.083549,0.081872,0.081947,0.082185,-0.073147,-0.073147,-0.073147,...,0.024619,0.024650,0.024650,0.024650,0.021856,0.021856,0.021856,0.006960,0.006960,0.006960
1,10759547,-0.080735,-0.080853,-0.080942,0.080371,0.080446,0.080683,-0.073147,-0.073147,-0.073147,...,0.009025,0.009082,0.009082,0.009082,0.007594,0.007594,0.007594,0.000977,0.000977,0.000977
2,10843547,-0.820392,-0.820510,-0.820599,1.631166,1.631242,1.631479,-0.092616,-0.092616,-0.092616,...,0.019134,0.019478,0.019478,0.019478,0.015242,0.015242,0.015242,0.016069,0.016069,0.016069
3,10859147,-0.113977,-0.114095,-0.114183,0.109437,0.109512,0.109749,-0.098430,-0.098430,-0.098430,...,0.027127,0.027727,0.027727,0.027727,0.020194,0.020194,0.020194,0.021430,0.021430,0.021430
4,10879947,-0.094550,-0.094668,-0.094757,0.091331,0.091406,0.091643,-0.083955,-0.083955,-0.083955,...,0.000500,0.000536,0.000536,0.000536,-0.000752,-0.000752,-0.000752,-0.002293,-0.002293,-0.002293


In [13]:
for model_pair in models:
    model_name, single_stage_model = model_pair
    
    new_submission = DataFrame({'ParcelId': test_predictions['parcelid'],
                           '201610':test_predictions['%s_10' % model_name],
                           '201611':test_predictions['%s_11' % model_name],
                           '201612':test_predictions['%s_12' % model_name],
    })
    new_submission['201710'] = 0
    new_submission['201711'] = 0
    new_submission['201712'] = 0
    
    print("%s:" % (model_name))
    print(mean_absolute_errors(new_submission, data))
    print("")
    new_submission.round(4).to_csv("/home/anerdi/Desktop/Zillow/submissions/\
    two_stage_stage1_stacked_annrfsxgbslgbms_stage2_%s_age.csv.gz" % model_name, index=False,
                 compression='gzip')
        
    print("%s using single-stage model:" % (model_name))
    print(mean_absolute_errors(single_stage_model, data))
    print("")

ridge:
(0.06126701975934998, 0.0603308609270727, 0.07293366428916853, 0.063442021961365447)

ridge using single-stage model:
(0.06332056539983931, 0.06250535370657166, 0.07484831374353078, 0.065493145336361525)

lasso:
(0.06129226551503985, 0.06025910691520667, 0.07301088748453059, 0.063457114028461645)

lasso using single-stage model:
(0.06333524733775354, 0.06244184118291356, 0.07476208625646924, 0.065470568485132233)

enet:
(0.06127988541786473, 0.060263641885242775, 0.07298179084920925, 0.063444946627714924)

enet using single-stage model:
(0.06332380630902144, 0.06246366413472067, 0.07474843300747566, 0.0654657878377429)

larm:
(0.06141419970429147, 0.06039021607414969, 0.07315946811582706, 0.063586434269852388)

larm using single-stage model:
(0.06345773598553339, 0.06251769879518071, 0.07483130304772855, 0.065572243736829725)

huber:
(0.06283620102796471, 0.061853516992334115, 0.07435472145282601, 0.064971102218525742)

huber using single-stage model:
(0.06360898332328704, 0.062